# **East Africa drought study - analysis of model data**

_Model fitting is less stable when using lognormal - carry out transformation separately_

In [5]:
source("../wwa_nonstationary_fitting.r")

---
## **Prep & testing**

In [2]:
# load data, fix values for factual & counterfactual climate

df <- read.csv("data/gmst-nino-cpc.csv")

gmst_2022 <- df[df$year == 2022, "gmst"]
gmst_cf <- gmst_2022 - 1.2

ondnino_2022 <- df[df$year == 2022, "nino_ond"]
ondnino_cf <- ondnino_2022

In [75]:
# wrapper function to get bootstrapped confidence intervals for spreadsheet
bootstrap_obs_results <- function(mdl, cov1, cov2, cov1_cf = NA, cov2_cf = NA, seed = 101, nsamp = 1000, dp = 5, convert_logs = T) {

    # create internal function to give required results
    get_mdl_values <- function(mdl, ev, cov1, cov2, cov1_cf = NA, cov2_cf = NA) {
        
        pars <- mdl$par
        
        rp <- 1/map_to_u(mdl, x = event_value, cov1 = cov1, cov2 = cov2)
        rp_cf <- 1/map_to_u(mdl, x = event_value, cov1 = cov1_cf, cov2 = cov2_cf)
        rl_cf <- map_from_u(1/rp, mdl, cov1 = cov1_cf, cov2 = cov2_cf)
        
        # if using log values, convert back to natural units
        if(convert_logs) {
            if(substr(mdl$varnm,1,5) == "log10") {
                ev <- 10^ev
                rl_cf <- 10^rl_cf
            } else if(substr(mdl$varnm,1,3) == "log") {
                ev <- exp(ev)
                rl_cf <- exp(rl_cf)
            }
        }
        c(pars, 
          "dispersion" = unname(pars["sigma0"] / pars["mu0"]),
          "event_magnitude" = ev,
          "return_period" = rp,
          "probability_ratio" = rp_cf/rp,
          "abs_change_in_intensity" = (ev - rl_cf),
          "rel_change_in_intensity" = ((ev - rl_cf) / rl_cf) * 100
         )   
    }
    
    # get best estimate from the observed data
    event_value <- mdl$x[mdl$ev_idx]
    mdl_res <- get_mdl_values(mdl, event_value, cov1, cov2, cov1_cf, cov2_cf)
    mdl_df <- setNames(data.frame(mdl$x, mdl$cov1, mdl$cov2), c(mdl$varnm, mdl$covnm_1, mdl$covnm_2)) 
    
    # get bootstrap sample
    set.seed(seed)    
    boot_res <- sapply(1:nsamp, function(i) {
        boot_df <- mdl_df[sample(1:nrow(mdl_df), nrow(mdl_df), replace = T),]
        boot_mdl <- fit_ns(mdl$dist, mdl$type, boot_df, varnm = mdl$varnm, covnm_1 = mdl$covnm_1, covnm_2 = mdl$covnm_2, lower = mdl$lower)
        get_mdl_values(boot_mdl, event_value, cov1, cov2, cov1_cf, cov2_cf)
    })
    boot_qq <- t(rbind("bestimate" = mdl_res, apply(boot_res, 1, quantile, c(0.025, 0.975), na.rm = T)))
    if(!is.na(dp)) boot_qq <- round(boot_qq, dp)
    return(boot_qq)
}

#### **Check results against Climate Explorer**

In [79]:
# fit test model to log precip - should give same results as Climate Explorer
bootstrap_obs_results(fit_ns("norm", "fixeddisp", df, varnm = "log10pr_ond", covnm_1 = "gmst", lower = T),
                        cov1 = gmst_2022, cov2 = ondnino_2022, cov1_cf = gmst_cf, cov2_cf = ondnino_cf, convert_logs = F)

,bestimate,2.5%,97.5%
mu0,2.11385,1.99395,2.26001
sigma0,0.19262,0.14105,0.24037
alpha,0.07560,-0.16343,0.27828
beta,0.00000,0.00000,0.00000
dispersion,0.09112,0.06683,0.11084
event_magnitude,1.93781,1.93781,1.93781
return_period,9.27779,4.73671,36.10080
probability_ratio,0.51682,0.06253,4.53706
abs_change_in_intensity,0.08141,-0.17479,0.30212
rel_change_in_intensity,4.38518,-8.27384,18.47067


In [80]:
# test same model using lognormal fit - should give comparable disp/RP/PR
bootstrap_obs_results(fit_ns("lnorm", "fixeddisp", df, varnm = "pr_ond", covnm_1 = "gmst", lower = T),
                  cov1 = gmst_2022, cov2 = ondnino_2022, cov1_cf = gmst_cf, cov2_cf = ondnino_cf, convert_logs = F)

,bestimate,2.5%,97.5%
mu0,4.86711,-8.581000e-02,5.33036
sigma0,0.44359,3.267500e-01,12.90564
alpha,0.17449,-6.056800e-01,0.77568
beta,0.00000,0.000000e+00,0.00000
dispersion,0.09114,-3.579750e+00,0.42307
event_magnitude,86.65775,8.665775e+01,86.65775
return_period,9.27930,1.118030e+00,35.59413
probability_ratio,0.51613,6.257000e-02,10.59477
abs_change_in_intensity,14.84415,-6.558811e+14,43.76041
rel_change_in_intensity,20.67039,-1.000000e+02,102.39797


#### **Instability in lognormal fit**

_Some instability in the model fit when using lognormal distribution: prefer to use log_pr & convert_

In [76]:
bootstrap_obs_results(fit_ns("lnorm", "fixeddisp", df, varnm = "pr_ond", covnm_1 = "gmst", covnm_2 = "nino_ond", lower = T),
                  cov1 = gmst_2022, cov2 = ondnino_2022, cov1_cf = gmst_cf, cov2_cf = ondnino_cf, convert_logs = T)

,bestimate,2.5%,97.5%
mu0,4.80534,-0.00591,4.601593e+01
sigma0,0.38312,0.30291,1.835417e+02
alpha,0.31110,-148.25073,3.508790e+00
beta,0.18709,-7.72621,2.179950e+00
dispersion,0.07973,-7.74113,1.420148e+06
event_magnitude,86.65775,86.65775,8.665775e+01
return_period,7.26351,1.03753,2.706110e+01
probability_ratio,0.31708,0.06809,1.336318e+02
abs_change_in_intensity,24.57644,-Inf,6.858379e+01
rel_change_in_intensity,39.58751,-100.00000,4.481868e+02


In [77]:
# pr_ond ~ gmst + nino_ond
bootstrap_obs_results(fit_ns("norm", "fixeddisp", df, varnm = "log10pr_ond", covnm_1 = "gmst", covnm_2 = "nino_ond", lower = T),
                  cov1 = gmst_2022, cov2 = ondnino_2022, cov1_cf = gmst_cf, cov2_cf = ondnino_cf, convert_logs = T)

,bestimate,2.5%,97.5%
mu0,2.08734,1.96873,2.20702
sigma0,0.16650,0.12496,0.19207
alpha,0.13447,-0.07215,0.32123
beta,0.07967,0.02650,0.12575
dispersion,0.07977,0.05895,0.09303
event_magnitude,86.65774,86.65774,86.65774
return_period,7.35726,3.69988,30.59830
probability_ratio,0.31701,0.06059,2.08681
abs_change_in_intensity,24.47894,-17.13128,47.55208
rel_change_in_intensity,39.36863,-16.50587,121.60260


In [78]:
# confirm: if we use natural log, we get almost same fitted parameters as using lognormal, but without the model instability
df["lognpr_ond"] <- log(df$pr_ond)
bootstrap_obs_results(fit_ns("norm", "fixeddisp", df, varnm = "lognpr_ond", covnm_1 = "gmst", covnm_2 = "nino_ond", lower = T),
                  cov1 = gmst_2022, cov2 = ondnino_2022, cov1_cf = gmst_cf, cov2_cf = ondnino_cf, convert_logs = T)

,bestimate,2.5%,97.5%
mu0,4.80631,4.53320,5.08185
sigma0,0.38336,0.28774,0.44230
alpha,0.30973,-0.16611,0.74022
beta,0.18346,0.06101,0.28957
dispersion,0.07976,0.05895,0.09304
event_magnitude,86.65775,86.65775,86.65775
return_period,7.36042,3.70003,30.59794
probability_ratio,0.31685,0.06059,2.08684
abs_change_in_intensity,24.48457,-17.13121,47.57549
rel_change_in_intensity,39.38124,-16.50581,121.73512


---
## **Observations**

In [82]:
source("../wwa_nonstationary_fitting.r")

# wrapper function to get bootstrapped confidence intervals for spreadsheet
bootstrap_obs_results <- function(mdl, cov1, cov2, cov1_cf = NA, cov2_cf = NA, seed = 101, nsamp = 1000, dp = 5, convert_logs = T, spreadsheet_format = T) {

    # create internal function to give required results
    get_mdl_values <- function(mdl, ev, cov1, cov2, cov1_cf = NA, cov2_cf = NA) {
        
        pars <- mdl$par
        
        rp <- 1/map_to_u(mdl, x = event_value, cov1 = cov1, cov2 = cov2)
        rp_cf <- 1/map_to_u(mdl, x = event_value, cov1 = cov1_cf, cov2 = cov2_cf)
        rl_cf <- map_from_u(1/rp, mdl, cov1 = cov1_cf, cov2 = cov2_cf)
        
        # if using log values, convert back to natural units
        if(convert_logs) {
            if(substr(mdl$varnm,1,5) == "log10") {
                ev <- 10^ev
                rl_cf <- 10^rl_cf
            } else if(substr(mdl$varnm,1,3) == "log") {
                ev <- exp(ev)
                rl_cf <- exp(rl_cf)
            }
        }
        c(pars, 
          "dispersion" = unname(pars["sigma0"] / pars["mu0"]),
          "event_magnitude" = ev,
          "return_period" = rp,
          "probability_ratio" = rp_cf/rp,
          "abs_change_in_intensity" = (ev - rl_cf),
          "rel_change_in_intensity" = ((ev - rl_cf) / rl_cf) * 100
         )   
    }
    
    # get best estimate from the observed data
    event_value <- mdl$x[mdl$ev_idx]
    mdl_res <- get_mdl_values(mdl, event_value, cov1, cov2, cov1_cf, cov2_cf)
    mdl_df <- setNames(data.frame(mdl$x, mdl$cov1, mdl$cov2), c(mdl$varnm, mdl$covnm_1, mdl$covnm_2)) 
    
    # get bootstrap sample
    set.seed(seed)    
    boot_res <- sapply(1:nsamp, function(i) {
        boot_df <- mdl_df[sample(1:nrow(mdl_df), nrow(mdl_df), replace = T),]
        boot_mdl <- fit_ns(mdl$dist, mdl$type, boot_df, varnm = mdl$varnm, covnm_1 = mdl$covnm_1, covnm_2 = mdl$covnm_2, lower = mdl$lower)
        get_mdl_values(boot_mdl, event_value, cov1, cov2, cov1_cf, cov2_cf)
    })
    boot_qq <- t(rbind("bestimate" = mdl_res, apply(boot_res, 1, quantile, c(0.025, 0.975), na.rm = T)))
    if(!is.na(dp)) boot_qq <- round(boot_qq, dp)
    
    if(spreadsheet_format) {
        return(t(data.frame("." = c(setNames(boot_qq["dispersion",], c("disp_est", "disp_lower", "disp_upper")),
                                    "event_magnitude" = boot_qq["event_magnitude", "bestimate"],
                                    setNames(boot_qq["return_period",], c("rp_est", "rp_lower", "rp_upper")),
                                    setNames(boot_qq["probability_ratio",], c("pr_est", "pr_lower", "pr_upper")),
                                    setNames(boot_qq["rel_change_in_intensity",], c("rel_DI_est", "rel_DI_lower", "rel_DI_upper")),
                                    setNames(boot_qq["abs_change_in_intensity",], c("abs_DI_est", "abs_DI_lower", "abs_DI_upper"))))))
    } else {
        return(boot_qq)
    }
}

### **CPC**

In [83]:
# load data, fix values for factual & counterfactual climate

df <- read.csv("data/gmst-nino-cpc.csv")

gmst_2022 <- df[df$year == 2022, "gmst"]
gmst_cf <- gmst_2022 - 1.2

ondnino_2022 <- df[df$year == 2022, "nino_ond"]
ondnino_cf <- ondnino_2022

In [87]:
# pr_ond ~ gmst + nino_ond
bootstrap_obs_results(fit_ns("norm", "fixeddisp", df, varnm = "log10pr_ond", covnm_1 = "gmst", covnm_2 = "nino_ond", lower = T),
                  cov1 = gmst_2022, cov2 = ondnino_2022, cov1_cf = gmst_cf, cov2_cf = ondnino_cf, convert_logs = T)

,disp_est,disp_lower,disp_upper,event_magnitude,rp_est,rp_lower,rp_upper,pr_est,pr_lower,pr_upper,rel_DI_est,rel_DI_lower,rel_DI_upper,abs_DI_est,abs_DI_lower,abs_DI_upper
.,0.07977,0.05895,0.09303,86.65774,7.35726,3.69988,30.5983,0.31701,0.06059,2.08681,39.36863,-16.50587,121.6026,24.47894,-17.13128,47.55208


In [86]:
# pr_mam ~ gmst
bootstrap_obs_results(fit_ns("norm", "fixeddisp", df, varnm = "log10pr_mam", covnm_1 = "gmst", lower = T),
                  cov1 = gmst_2022, cov2 = ondnino_2022, cov1_cf = gmst_cf, cov2_cf = ondnino_cf, convert_logs = T)

,disp_est,disp_lower,disp_upper,event_magnitude,rp_est,rp_lower,rp_upper,pr_est,pr_lower,pr_upper,rel_DI_est,rel_DI_lower,rel_DI_upper,abs_DI_est,abs_DI_lower,abs_DI_upper
.,0.06466,0.04878,0.07624,100.6288,10.88907,4.36724,52.71624,6.93885,0.24862,3622.234,-26.63637,-59.90816,24.83328,-36.53563,-150.3674,20.01821


In [88]:
# pr_24 ~ gmst
bootstrap_obs_results(fit_ns("norm", "fixeddisp", df[-1,], varnm = "log10pr24", covnm_1 = "gmst", lower = T),
                  cov1 = gmst_2022, cov2 = ondnino_2022, cov1_cf = gmst_cf, cov2_cf = ondnino_cf, convert_logs = T)

,disp_est,disp_lower,disp_upper,event_magnitude,rp_est,rp_lower,rp_upper,pr_est,pr_lower,pr_upper,rel_DI_est,rel_DI_lower,rel_DI_upper,abs_DI_est,abs_DI_lower,abs_DI_upper
.,0.02798,0.02122,0.03282,588.2868,21.47824,8.05635,135.6907,2.27386,0.0904,361.3354,-6.6726,-31.25869,21.18943,-42.06057,-267.5113,102.8589


In [89]:
# pet_24 ~ gmst
bootstrap_obs_results(fit_ns("norm", "shift", df[-1,], varnm = "pet_24", covnm_1 = "gmst", lower = F),
                  cov1 = gmst_2022, cov2 = ondnino_2022, cov1_cf = gmst_cf, cov2_cf = ondnino_cf, dp = 3)

,disp_est,disp_lower,disp_upper,event_magnitude,rp_est,rp_lower,rp_upper,pr_est,pr_lower,pr_upper,rel_DI_est,rel_DI_lower,rel_DI_upper,abs_DI_est,abs_DI_lower,abs_DI_upper
.,0.042,0.033,0.048,121.413,2.659,1.71,5.398,1030128,2996.915,452872825077,20.085,12.697,29.446,20.307,13.679,27.619


### **CHIRPS / CenTrends**

In [90]:
# load data, fix values for factual & counterfactual climate

df <- read.csv("data/gmst-nino-centrends.csv")

gmst_2022 <- df[df$year == 2022, "gmst"]
gmst_cf <- gmst_2022 - 1.2

ondnino_2022 <- df[df$year == 2022, "nino_ond"]
ondnino_cf <- ondnino_2022

In [91]:
# pr_ond ~ gmst + nino_ond
bootstrap_obs_results(fit_ns("norm", "fixeddisp", df, varnm = "log10pr_ond", covnm_1 = "gmst", covnm_2 = "nino_ond", lower = T),
                  cov1 = gmst_2022, cov2 = ondnino_2022, cov1_cf = gmst_cf, cov2_cf = ondnino_cf, convert_logs = T)

,disp_est,disp_lower,disp_upper,event_magnitude,rp_est,rp_lower,rp_upper,pr_est,pr_lower,pr_upper,rel_DI_est,rel_DI_lower,rel_DI_upper,abs_DI_est,abs_DI_lower,abs_DI_upper
.,0.06751,0.0571,0.0763,131.7356,2.86545,1.8672,5.24158,0.52184,0.23563,1.10065,30.20158,-3.91918,69.88559,30.55741,-5.37356,54.19189


In [92]:
# pr_mam ~ gmst
bootstrap_obs_results(fit_ns("norm", "fixeddisp", df, varnm = "log10pr_mam", covnm_1 = "gmst", lower = T),
                  cov1 = gmst_2022, cov2 = ondnino_2022, cov1_cf = gmst_cf, cov2_cf = ondnino_cf, convert_logs = T)

,disp_est,disp_lower,disp_upper,event_magnitude,rp_est,rp_lower,rp_upper,pr_est,pr_lower,pr_upper,rel_DI_est,rel_DI_lower,rel_DI_upper,abs_DI_est,abs_DI_lower,abs_DI_upper
.,0.05606,0.04807,0.06233,144.8896,5.91954,3.03041,15.04343,1.6057,0.38096,7.4167,-8.44292,-30.46315,17.1441,-13.36097,-63.47415,21.20465


In [93]:
# pr_24 ~ gmst
bootstrap_obs_results(fit_ns("norm", "fixeddisp", df[-1,], varnm = "log10pr24", covnm_1 = "gmst", lower = T),
                  cov1 = gmst_2022, cov2 = ondnino_2022, cov1_cf = gmst_cf, cov2_cf = ondnino_cf, convert_logs = T)

,disp_est,disp_lower,disp_upper,event_magnitude,rp_est,rp_lower,rp_upper,pr_est,pr_lower,pr_upper,rel_DI_est,rel_DI_lower,rel_DI_upper,abs_DI_est,abs_DI_lower,abs_DI_upper
.,0.02475,0.0215,0.02733,707.1602,19.31896,7.84467,68.97395,0.68806,0.11,4.51354,3.25257,-11.78421,19.15895,22.27632,-94.46519,113.7006


---
## **Models**

In [81]:
spreadsheet_results <- function(df, dist, fittype, varnm, covnm_1, covnm_2 = NA, lower = F, rp = 10, delta_relative = T, nsamp = 5, seed = 1) {
    
    # FUNCTION TO CARRY OUT ALL ATTRIBUTION RUNS & PRODUCE RESULTS IN SPREADSHEET-FRIENDLY FORM
    
    set.seed(seed)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # FIT MODEL to three different subsets: for evaluation, attribution & projection
    
    df_eval <- df[df$year >= 1979 & df$year <= 2022,]
    df_attr <- df[df$year <= 2022,]
    df_proj <- df[df$year <= 2050,]
    
    mdl_eval <- fit_ns(dist, fittype, df_eval, varnm = varnm, covnm_1 = covnm_1, covnm_2 = covnm_2, lower = lower)
    mdl_attr <- fit_ns(dist, fittype, df_attr, varnm = varnm, covnm_1 = covnm_1, covnm_2 = covnm_2, lower = lower)
    mdl_proj <- fit_ns(dist, fittype, df_proj, varnm = varnm, covnm_1 = covnm_1, covnm_2 = covnm_2, lower = lower)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    ## EVALUATION (get model parameters)
    
    disp_eval <- mdl_eval$par["sigma0"] / mdl_eval$par["mu0"]
    boot_eval <- sapply(1:nsamp, function(i) {
        
        boot_df <- df_eval[sample(1:nrow(df_eval), nrow(df_eval), replace = T),]
        boot_mdl <- fit_ns(mdl_eval$dist, mdl_eval$type, boot_df, varnm = mdl_eval$varnm, covnm_1 = mdl_eval$covnm_1, covnm_2 = mdl_eval$covnm_2, lower = mdl_eval$lower)
        boot_mdl$par["sigma0"] / boot_mdl$par["mu0"]
    })
    ci_eval <- setNames(c(disp_eval, quantile(boot_eval, c(0.025, 0.975))),
                        c("disp_est", "disp_lower", "disp_upper"))
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    ## ATTRIBUTION (get event value, PR & DI)
    
    logrl_2022 <- map_from_u(1/rp, mdl_attr, cov1 = gsat_2022, cov2 = nino_2022)
    logrl_cf <- map_from_u(1/rp, mdl_attr, cov1 = gsat_hist, cov2 = nino_cf)
    
    rl_2022 <- 10^logrl_2022
    
    pr_eval <- prob_ratio(mdl_attr, logrl_2022, cov1 = gsat_2022, cov1_cf = gsat_hist, cov2 = nino_2022, cov2_cf = nino_cf)
    dI_eval <- Delta_I(mdl_attr, rp, cov1 = gsat_2022, cov1_cf = gsat_hist, relative = delta_relative)
    
    boot_attr <- sapply(1:nsamp, function(i) {
        
        boot_df <- df_attr[sample(1:nrow(df_attr), nrow(df_attr), replace = T),]
        boot_mdl <- fit_ns(mdl_attr$dist, mdl_attr$type, boot_df, varnm = mdl_attr$varnm, covnm_1 = mdl_attr$covnm_1, covnm_2 = mdl_attr$covnm_2, lower = mdl_attr$lower)
        
        boot_pr <- prob_ratio(boot_mdl, logrl_2022, cov1 = gsat_2022, cov1_cf = gsat_hist, cov2 = nino_2022, cov2_cf = nino_cf)
        boot_di <- Delta_I(boot_mdl, 10, cov1 = gsat_2022, cov1_cf = gsat_hist, cov2 = nino_2022, cov2_cf = nino_cf, relative = delta_relative)
        
        c("PR" = boot_pr, "DI" = boot_di)
    })
    ci_attr <- rbind("est" = c(pr_eval, dI_eval), apply(boot_attr, 1, quantile, c(0.025, 0.975)))
    ci_attr <- unlist(lapply(colnames(ci_attr), function(cnm) setNames(ci_attr[,cnm], paste("attr", cnm, c("est", "lower", "upper"), sep = "_"))))
                          
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    ## PROJECTION (get PR & DI)

    pr_proj <- prob_ratio(mdl_proj, logrl_2022, cov1 = gsat_2022, cov1_cf = gsat_fut, cov2 = nino_2022, cov2_cf = nino_cf)
    dI_proj <- Delta_I(mdl_proj, 10, cov1 = gsat_2022, cov1_cf = gsat_fut, relative = delta_relative)
                             
    boot_proj <- sapply(1:nsamp, function(i) {
        
        boot_df <- df_proj[sample(1:nrow(df_proj), nrow(df_proj), replace = T),]
        boot_mdl <- fit_ns(mdl_proj$dist, mdl_proj$type, boot_df, varnm = mdl_proj$varnm, covnm_1 = mdl_proj$covnm_1, covnm_2 = mdl_proj$covnm_2, lower = mdl_proj$lower)
        
        boot_pr <- prob_ratio(boot_mdl, logrl_2022, cov1 = gsat_2022, cov1_cf = gsat_fut, cov2 = nino_2022, cov2_cf = nino_cf)
        boot_di <- Delta_I(boot_mdl, 10, cov1 = gsat_2022, cov1_cf = gsat_fut, cov2 = nino_2022, cov2_cf = nino_cf, relative = delta_relative)
        
        c("PR" = boot_pr, "DI" = boot_di)
    })
    ci_proj <- rbind("est" = c(pr_proj, dI_proj), apply(boot_proj, 1, quantile, c(0.025, 0.975)))
             
    # invert
    ci_proj[,"PR"] <- 1/ci_proj[c(1,3,2),"PR"]
    ci_proj[,"DI"] <- -ci_proj[c(1,3,2),"DI"]   
                             
    ci_proj <- unlist(lapply(colnames(ci_proj), function(cnm) setNames(ci_proj[,cnm], paste("proj", cnm, c("est", "lower", "upper"), sep = "_"))))
                             
    # #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                             
    t(setNames(data.frame(c(ci_eval, "rp_value" = rl_2022, ci_attr, ci_proj)), gsub(".dat", "", gsub(".+pr-mam_", "", fnm))))                  
}



,disp_est,disp_lower,disp_upper,rp_value,attr_PR_est,attr_PR_lower,attr_PR_upper,attr_DI_est,attr_DI_lower,attr_DI_upper,proj_PR_est,proj_PR_lower,proj_PR_upper,proj_DI_est,proj_DI_lower,proj_DI_upper
AFR-22_CanESM2_r1_CanRCM4,0.04482604,0.03541999,0.05265941,128.9629,0.8721586,0.5805268,1.051945,1.951585,-0.8462723,9.681931,0.6398009,0.4051871,0.7001337,5.25442,4.072715,8.125979


### **CORDEX**

In [138]:
source("../wwa_nonstationary_fitting.r")

In [141]:
cordex_res <- sapply(fl, function(fnm) {
    if(fnm %in% c("ts/EA-drought_pr-mam_AFR-44_HadGEM2-ES_r1_RegCM4-3.dat")) {return(NULL)}
    
    gsat_fnm <- list.files("../00_gsat", pattern = paste0(strsplit(fnm, "_")[[1]][4],"_rcp85_",strsplit(fnm, "_")[[1]][5],"i"), full.names = T)
    if(length(gsat_fnm) == 0) {
        cat("No GSAT:", paste0(strsplit(fnm, "_")[[1]][4],"_rcp85_",strsplit(fnm, "_")[[1]][5]), "\n")
        return(NULL)
    } else {
        # load necessary data
        gmst <- read.csv(gsat_fnm, comment.char = "#", sep = " ", header = F, col.names = c("year", "gmst"))
        pr <- read.csv(fnm, comment.char = "#", sep = " ", header = F, col.names = c("year", "pr"))
        df <- merge(gsat, pr)
        df["log10pr"] <- log10(df$pr)   # fit models to log precip, convert back at the end (more stable model fitting)
        
        # covariates defining current, past & future climates
        gsat_2022 <- gmst[gmst$year == 2022, "gmst"]
        gsat_hist <- gsat_2022 - 1.2
        gsat_fut <- gsat_2022 + 0.8
        
        nino_2022 <- nino_cf <- 0    # dummy variables for now
        
        data.frame(spreadsheet_results(df, "norm", "fixeddisp", varnm = "log10pr", covnm_1 = "gmst", lower = T, rp = 10, delta_relative = 10))
    }
}, simplify = F)
do.call("rbind", (cordex_res[!sapply(cordex_res, is.null)]))

No GSAT: GFDL-ESM2M_rcp85_r1 
No GSAT: IPSL-CM5A-MR_rcp85_r1 
No GSAT: MPI-ESM-LR_rcp85_r2 
No GSAT: MPI-ESM-LR_rcp85_r3 


,disp_est,disp_lower,disp_upper,rp_value,attr_PR_est,attr_PR_lower,attr_PR_upper,attr_DI_est,attr_DI_lower,attr_DI_upper,proj_PR_est,proj_PR_lower,proj_PR_upper,proj_DI_est,proj_DI_lower,proj_DI_upper
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ts/EA-drought_pr-mam_AFR-22_CanESM2_r1_CanRCM4.dat,0.04482604,0.03541999,0.05265941,1.289629e+02,0.8721586,0.5805268,1.0519450,1.9515850,-0.84627232,9.681931,0.6398009,0.4051871,0.7001337,5.2544203,4.0727145,8.1259786
ts/EA-drought_pr-mam_AFR-22_HadGEM2-ES_r1_CCLM5-0-15.dat,0.14766496,0.13749037,0.16477519,1.934342e+01,1.1344319,1.1652431,1.8666311,-3.3749244,-14.23938850,-4.382719,0.7805499,0.6290646,0.8622417,5.8809691,3.9534049,10.1944332
ts/EA-drought_pr-mam_AFR-22_HadGEM2-ES_r1_RegCM4-7.dat,0.10157657,0.08869735,0.13074500,4.517070e+01,1.5444237,0.7062755,2.5015535,-9.5187361,-17.68590874,6.042459,1.0904602,1.0274857,1.4659183,-1.7420883,-6.1754827,-0.6831292
ts/EA-drought_pr-mam_AFR-22_HadGEM2-ES_r1_REMO2015.dat,0.10447639,0.08390809,0.14088840,3.431283e+01,0.8044069,0.5339862,1.4878178,5.4871692,-8.20603544,21.070215,0.7327722,0.6209083,0.8821213,6.4185816,3.0634996,10.3687493
ts/EA-drought_pr-mam_AFR-22_MPI-ESM-LR_r1_CCLM5-0-15.dat,0.11082379,0.10494212,0.11513381,4.614764e+01,1.2565456,0.7248058,1.5013784,-5.9984944,-9.04825676,8.353350,1.1406012,1.0544899,1.2066420,-3.9836074,-6.3616696,-1.8048478
ts/EA-drought_pr-mam_AFR-22_MPI-ESM-LR_r1_REMO2015.dat,0.08173031,0.07218690,0.08812021,6.149987e+01,1.7731326,0.7142739,3.7372959,-11.1283438,-25.84498103,6.800526,1.2044917,0.9776564,1.4200935,-4.4894343,-10.2704429,0.5234793
ts/EA-drought_pr-mam_AFR-22_MPI-ESM-MR_r1_RegCM4-7.dat,0.07777680,0.07057627,0.07881665,8.865020e+01,0.7947183,0.3270481,1.0724448,5.6657550,-1.54124673,24.598992,1.1639829,0.9199101,1.1151219,-3.7906902,-2.5961328,1.7251075
ts/EA-drought_pr-mam_AFR-22_NorESM1-M_r1_CCLM5-0-15.dat,0.13086931,0.12134848,0.13904336,3.151063e+01,0.9064145,0.8331386,1.6952729,3.0033645,-12.11593028,5.222325,1.2188398,1.0487036,1.3637412,-6.6986833,-11.3517425,-1.5798394
ts/EA-drought_pr-mam_AFR-22_NorESM1-M_r1_RegCM4-7.dat,0.11117588,0.09496766,0.12179633,4.556881e+01,1.3487510,0.8881928,3.2331395,-7.7403777,-20.97127793,4.479616,1.2112779,0.9362749,1.3620891,-5.2907252,-8.3488111,1.8450595
